# 2D animation

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cdflib
from tqdm import tqdm

def load_vectors_from_cdf(cdf_path, step=50):
    try:
        cdf = cdflib.CDF(cdf_path)
        Bvec = cdf['OB_B']
        POSN = cdf['POSN']

        if Bvec.shape[0] == 0 or POSN.shape[0] == 0:
            return None, None

        Bvec_sub = Bvec[::step, :2]  # Bx, By
        POSN_sub = POSN[::step, :2]  # X, Y
        return POSN_sub, Bvec_sub
    except Exception as e:
        print(f"Error reading {cdf_path}: {e}")
        return None, None

def animate_vectors(data_folder, step=50, max_frames=1000):
    cdf_files = sorted(glob.glob(os.path.join(data_folder, '**/*.cdf'), recursive=True))
    print(f"Found {len(cdf_files)} CDF files.")

    if len(cdf_files) == 0:
        print("❌ No CDF files found.")
        return

    frame_files = cdf_files[:min(max_frames, len(cdf_files))]

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_xlim(-2e4, 2e4)
    ax.set_ylim(-2e4, 2e4)
    ax.set_xlabel("X (km)")
    ax.set_ylabel("Y (km)")
    ax.set_title("MAVEN Magnetic Field Vectors (2014–2025)")
    ax.grid(True)
    ax.set_aspect('equal')

    positions = []
    vectors = []

    print("📦 Processing CDF files...")
    for path in tqdm(frame_files, desc="Processing CDF files", unit="file"):
        posn, bvec = load_vectors_from_cdf(path, step)
        if posn is not None and bvec is not None:
            positions.append(posn)
            vectors.append(bvec)

    if not positions:
        print("❌ No valid data to plot.")
        return

    def update(frame_idx):
        ax.clear()
        ax.set_xlim(-2e4, 2e4)
        ax.set_ylim(-2e4, 2e4)
        ax.set_xlabel("X (km)")
        ax.set_ylabel("Y (km)")
        ax.set_title("MAVEN Magnetic Field Vectors (2014–2025)")
        ax.grid(True)
        ax.set_aspect('equal')

        X, Y = positions[frame_idx][:, 0], positions[frame_idx][:, 1]
        U, V = vectors[frame_idx][:, 0], vectors[frame_idx][:, 1]
        ax.quiver(X, Y, U, V, scale=1e4, width=0.002, color='blue')

    ani = animation.FuncAnimation(fig, update, frames=len(positions), interval=100)
    ani.save("maven_magnetic_vectors.mp4", writer='ffmpeg', dpi=150)
    print("✅ Saved: maven_magnetic_vectors.mp4")
    plt.close()

if __name__ == "__main__":
    data_folder = "maven_cdfs"  # Change this to your actual path
    animate_vectors(data_folder, step=50, max_frames=1000)


# 3D animation

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import cdflib
from tqdm import tqdm

MARS_RADIUS = 3390  # Mars radius in km

def load_vectors_from_cdf(cdf_path, step=50):
    try:
        cdf = cdflib.CDF(cdf_path)
        Bvec = cdf['OB_B']
        POSN = cdf['POSN']
        if Bvec.shape[0] == 0 or POSN.shape[0] == 0:
            return None, None
        return POSN[::step, :], Bvec[::step, :]
    except Exception as e:
        print(f"Error reading {cdf_path}: {e}")
        return None, None

def draw_mars(ax):
    # Create sphere
    u, v = np.mgrid[0:2 * np.pi:60j, 0:np.pi:30j]
    x = MARS_RADIUS * np.cos(u) * np.sin(v)
    y = MARS_RADIUS * np.sin(u) * np.sin(v)
    z = MARS_RADIUS * np.cos(v)

    ax.plot_surface(x, y, z, color='red', alpha=0.3, edgecolor='none')

def animate_vectors_3d(data_folder, step=50, max_frames=500):
    cdf_files = sorted(glob.glob(os.path.join(data_folder, '**/*.cdf'), recursive=True))
    print(f"Found {len(cdf_files)} CDF files.")

    if not cdf_files:
        print("❌ No CDF files found.")
        return

    frame_files = cdf_files[:min(max_frames, len(cdf_files))]

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    ax.set_xlim(-2e4, 2e4)
    ax.set_ylim(-2e4, 2e4)
    ax.set_zlim(-2e4, 2e4)
    ax.set_xlabel("X (km)")
    ax.set_ylabel("Y (km)")
    ax.set_zlabel("Z (km)")
    ax.set_title("MAVEN Magnetic Field Vectors (2014–2025)")
    ax.view_init(elev=30, azim=45)

    positions, vectors = [], []

    print("📦 Processing CDF files...")
    for path in tqdm(frame_files, desc="Processing CDF files", unit="file"):
        posn, bvec = load_vectors_from_cdf(path, step)
        if posn is not None and bvec is not None:
            positions.append(posn)
            vectors.append(bvec)

    if not positions:
        print("❌ No valid data to plot.")
        return

    def update(frame_idx):
        ax.cla()
        ax.set_xlim(-2e4, 2e4)
        ax.set_ylim(-2e4, 2e4)
        ax.set_zlim(-2e4, 2e4)
        ax.set_xlabel("X (km)")
        ax.set_ylabel("Y (km)")
        ax.set_zlabel("Z (km)")
        ax.set_title("MAVEN Magnetic Field Vectors (2014–2025)")
        ax.view_init(elev=30, azim=45)

        draw_mars(ax)  # 🌍 Add Mars globe

        pos = positions[frame_idx]
        vec = vectors[frame_idx]
        X, Y, Z = pos[:, 0], pos[:, 1], pos[:, 2]
        U, V, W = vec[:, 0], vec[:, 1], vec[:, 2]

        ax.quiver(X, Y, Z, U, V, W, length=500, normalize=True, color='blue', linewidth=0.5)

    ani = animation.FuncAnimation(fig, update, frames=len(positions), interval=100)
    ani.save("maven_magnetic_vectors_3d_with_mars.mp4", writer='ffmpeg', dpi=150)
    print("✅ Saved: maven_magnetic_vectors_3d_with_mars.mp4")
    plt.close()

if __name__ == "__main__":
    data_folder = "maven_cdfs"  # Your folder with CDF files
    animate_vectors_3d(data_folder, step=50, max_frames=500)


# MAVEN Map

In [ ]:
import cdflib
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm

def load_cdf_components(cdf_path):
    try:
        cdf = cdflib.CDF(cdf_path)
        Bvec = cdf['OB_B']     # Magnetic field vector components (nT)
        pos = cdf['POSN']      # Spacecraft position (km)

        if Bvec.shape[0] == 0 or pos.shape[0] == 0:
            return None

        Bx = Bvec[:, 0]
        By = Bvec[:, 1]
        Bz = Bvec[:, 2]
        Bmag = np.linalg.norm(Bvec, axis=1)

        r = np.linalg.norm(pos, axis=1)
        lat = np.degrees(np.arcsin(pos[:, 2] / r))
        lon = np.degrees(np.arctan2(pos[:, 1], pos[:, 0]))
        lon = (lon + 360) % 360  # Convert longitude to [0, 360)

        # Calculate radial component Br = (B · r̂)
        # Unit vector r̂ = pos / |pos|
        r_hat = pos / r[:, np.newaxis]  # shape (N, 3)
        Br = np.sum(Bvec * r_hat, axis=1)

        alt = r - 3390  # Mars radius ~3390 km

        return lat, lon, Bx, By, Bz, Br, Bmag, alt
    except Exception as e:
        print(f"Error loading {cdf_path}: {e}")
        return None

def main(data_folder):
    cdf_files = sorted(glob.glob(os.path.join(data_folder, '**/*.cdf'), recursive=True))
    print(f"Found {len(cdf_files)} CDF files.")

    all_lat, all_lon = [], []
    all_Bx, all_By, all_Bz, all_Br, all_Bmag = [], [], [], [], []

    for path in tqdm(cdf_files, desc="Processing CDF files", unit="file"):
        result = load_cdf_components(path)
        if result:
            lat, lon, Bx, By, Bz, Br, Bmag, alt = result

            # Filter data to altitude below 400 km
            mask = (alt < 400)
            all_lat.append(lat[mask])
            all_lon.append(lon[mask])
            all_Bx.append(Bx[mask])
            all_By.append(By[mask])
            all_Bz.append(Bz[mask])
            all_Br.append(Br[mask])
            all_Bmag.append(Bmag[mask])

    if not all_lat:
        print("❌ No valid data to plot.")
        return

    # Concatenate all data arrays
    lat = np.concatenate(all_lat)
    lon = np.concatenate(all_lon)
    Bx = np.concatenate(all_Bx)
    By = np.concatenate(all_By)
    Bz = np.concatenate(all_Bz)
    Br = np.concatenate(all_Br)
    Bmag = np.concatenate(all_Bmag)

    # Define bins for latitude and longitude
    lat_bins = np.linspace(-90, 90, 181)   # 1 degree bins
    lon_bins = np.linspace(0, 360, 361)    # 1 degree bins

    def binned_map(data):
        grid, _, _ = np.histogram2d(lat, lon, bins=[lat_bins, lon_bins], weights=data)
        counts, _, _ = np.histogram2d(lat, lon, bins=[lat_bins, lon_bins])
        return np.divide(grid, counts, out=np.zeros_like(grid), where=counts != 0)

    # Create binned maps
    Bx_map = binned_map(Bx)
    By_map = binned_map(By)
    Bz_map = binned_map(Bz)
    Br_map = binned_map(Br)
    Bmag_map = binned_map(Bmag)

    # Plotting helper
    def plot_map(data, title, cmap='coolwarm', vmin=None, vmax=None):
        fig, ax = plt.subplots(figsize=(12, 6))
        im = ax.imshow(
            data,
            extent=[0, 360, -90, 90],
            origin='lower',
            aspect='auto',
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(title)
        ax.set_xlabel("Longitude (deg)")
        ax.set_ylabel("Latitude (deg)")
        cbar = plt.colorbar(im, ax=ax, orientation='vertical')
        cbar.set_label('Magnetic Field (nT)')
        plt.tight_layout()
        plt.show()

    # Determine common color scale limits for components (symmetric for components)
    max_abs_component = max(
        np.nanmax(np.abs(Bx_map)),
        np.nanmax(np.abs(By_map)),
        np.nanmax(np.abs(Bz_map)),
        np.nanmax(np.abs(Br_map))
    )

    # Plot all maps
    plot_map(Bx_map, 'Mars Magnetic Field Bx Component Map (<400 km altitude)', cmap='inferno', vmin=-max_abs_component, vmax=max_abs_component)
    plot_map(By_map, 'Mars Magnetic Field By Component Map (<400 km altitude)', cmap='inferno', vmin=-max_abs_component, vmax=max_abs_component)
    plot_map(Bz_map, 'Mars Magnetic Field Bz Component Map (<400 km altitude)', cmap='inferno', vmin=-max_abs_component, vmax=max_abs_component)
    plot_map(Br_map, 'Mars Magnetic Field Radial Component Br Map (<400 km altitude)', cmap='inferno', vmin=-max_abs_component, vmax=max_abs_component)
    plot_map(Bmag_map, 'Mars Magnetic Field |B| Magnitude Map (<400 km altitude)', cmap='inferno')

if __name__ == "__main__":
    data_folder = "maven_cdfs"  # Change to your folder containing CDF files
    main(data_folder)
